# Configuration System

RADICAL-Pilot (RP) uses a configuration system to set control and management parameters for the initialization of its components and to define resource entry points for the target platform.

It includes:

* [Run description](#Run-description)
  * Resource label for a target platform configuration file;
  * Project allocation name (i.e., account/project) - _specific for HPC platforms_;
  * Job queue name (i.e., queue/partition) - _specific for HPC platforms_;
  * Amount of the resources (e.g., `cores`, `gpus`, `memory`) to allocate for the runtime period;
  * Mode to access the target platform (e.g., `local`, `ssh`) - _optional, default is "local"_.
* [Target platform description](#Platform-description)
  * Batch system (e.g., `SLURM`, `LSF`, etc.);
  * Provided launch methods (e.g., `SRUN`, `MPIRUN`, etc.);
  * Environment setup (including package manager, working directory, etc.);
  * Entry points: batch system URL, file system URL.

## Run description

Users have to describe at least one pilot in each RP application. That is done by instantiating a :class:`rp.PilotDescription <radical.pilot.PilotDescription>` object. Among that object's attributes, `pd.resource` is mandatory and is referred as a resource label (or platform ID), which corresponds to a target platform configuration file (see the section [Platform description](#Platform-description)). Users need to know what ID corresponds to the HPC platform on which they want to execute their RP application.

### Allocation parameters

Every run should state the project name (i.e., allocation account), preferable queue for a job submission, and the amount of required resources explicitly, unless it is a run on _localhost_ without accessing any batch system.

```python
import radical.pilot as rp

pd = rp.PilotDescription({
    'resource': 'ornl.frontier',  # platform ID
    'project' : 'XYZ000',         # allocation account
    'queue'   : 'debug',          # optional (default value is in the platform description)
    'cores'   : 32,               # amount of CPU slots
    'gpus'    : 8,                # amount of GPU slots
    'runtime' : 15                # maximum runtime for a pilot (in minutes)
})
```

### Resource access schema

Resource access schema (`pd.access_schema`) defines a set of endpoints for job submission and file system access. It is provided as part of a platform description, and in case of more than one access schemas users can set a specific one in :class:`rp.PilotDescription <radical.pilot.PilotDescription>`. Check schema availability per target platform:

* Launching RP application **from the target platform**:
  * `local` (_default_) - allows to run application from login nodes of the specific machine, compute nodes while within the interactive session, or within a batch script.
* Launching RP application **outside the target platform**:
  * `ssh` - use SSH protocol and corresponding SSH client to access the platform remotely.
  * `gsissh` - use GSI-enabled SSH to access the platform remotely.

<div class="alert alert-warning">

__Warning:__ Most platforms do not allow remote access for job submissions, thus this parameter shouldn't be set by user and default value should be used. For details on submission of applications on HPC see the tutorial [Using RADICAL-Pilot on HPC Platforms](submission.ipynb).

</div>

## Platform description

The RADICAL-Pilot uses configuration files for bookkeeping of supported platforms. Each configuration file identifies a facility (e.g., ACCESS, TACC, ORNL, ANL, etc.), is written in JSON and is named following the `resource_<facility_name>.json` convention. Each facility configuration file contains a set of platform names/labels with corresponding configuration parameters. Resource label (or platform ID) follows the `<facility_name>.<platform_name>` convention, and users use it for the `pd.resource` attribute of their :class:`rp.PilotDescription <radical.pilot.PilotDescription>` object.

### Predefined configurations

The RADICAL-Pilot development team maintains a growing set of pre-defined configuration files for supported HPC platforms (list platform descriptions in RP's [GitHub repo](https://github.com/radical-cybertools/radical.pilot/tree/master/src/radical/pilot/configs)).

For example, if users want to execute their RP application on Frontera, they will have to search for the [resource_tacc.json](https://github.com/radical-cybertools/radical.pilot/blob/master/src/radical/pilot/configs/resource_tacc.json) file and, inside that file, for the key(s) that start with the name `frontera`. The file `resource_tacc.json` contains the keys `frontera`, `frontera_rtx`, and `frontera_prte`. Each key identifies a specific set of configuration parameters: `frontera` offers a general-purpose set of configuration parameters; `frontera_rtx` enables the use of the `rtx` queue for GPU nodes; and `frontera_prte` enables the use of the PRRTE-based launch method to execute the application's tasks. Thus, for Frontera, the value for `resource` will be `tacc.frontera`, `tacc.frontera_rtx` or `tacc.frontera_prte`.  

### Customizing a predefined configuration

Users can customize existing platform configuration files by overwriting existing key/value pairs with ones from configuration files, which have the same names, but located in a user space. Default location of user-defined configuration files is `$HOME/.radical/pilot/configs/`. 

<div class="alert alert-info">

__Note:__ To change the location for user-defined platform configuration files, please, use env variable `RADICAL_CONFIG_USER_DIR`, which will be used instead of env variable `HOME` in the location path above. Make sure that the corresponding path exists, before creating configs there.

</div>

Two examples of customized configurations are below: (i) in one for `ornl.frontier` you change parameter __system_architecture.options__, and (ii) in another for `tacc.frontera` you set a default launch method `MPIEXEC`. With that files, every pilot description using `'resource': 'ornl.frontier'` or `'resource': 'tacc.frontera'` would use that new values. Changed parameters are described in the following section.

__resource_ornl.json__
```json
{
    "frontier": {
        "system_architecture": {
            "options": ["nvme"]
        }
    }
}
```

__resource_tacc.json__
```json
{
    "frontera": {
        "launch_methods": {
            "order"  : ["MPIEXEC"],
            "MPIEXEC": {}
        }
    }
}
```

### User-defined configuration

Users can write whole new configuration for an existing or a new platform with arbitrary platform ID. For example, you will create a custom platform configuration entry `resource_tacc.json` locally. That file will be loaded into the :class:`rp.Session <radical.pilot.Session>` object alongside with other configurations for TACC-related platforms.

In [ ]:
resource_tacc_tutorial = \
{
    "frontera_tutorial":
    {
        "description"                 : "Short description of the resource",
        "notes"                       : "Notes about resource usage",

        "default_schema"              : "local",
        "schemas"                     : {
            "local"                   : {
                "job_manager_endpoint": "slurm://frontera.tacc.utexas.edu/",
                "filesystem_endpoint" : "file://frontera.tacc.utexas.edu/"
            },
            "ssh"                     : {
                "job_manager_endpoint": "slurm+ssh://frontera.tacc.utexas.edu/",
                "filesystem_endpoint" : "sftp://frontera.tacc.utexas.edu/"
            },
            "no_submission"           : {
                "job_manager_endpoint": "fork://localhost/",
                "filesystem_endpoint" : "file://localhost/"
            }
        },

        "default_queue"               : "production",
        "resource_manager"            : "SLURM",

        "cores_per_node"              : 56,
        "gpus_per_node"               : 0,
        "system_architecture"         : {
                                         "smt"           : 1,
                                         "options"       : ["nvme", "intel"],
                                         "blocked_cores" : [],
                                         "blocked_gpus"  : []
                                        },

        "agent_config"                : "default",
        "agent_scheduler"             : "CONTINUOUS",
        "agent_spawner"               : "POPEN",
        "default_remote_workdir"      : "$HOME",

        "pre_bootstrap_0"             : [
                                         "module unload intel impi",
                                         "module load   intel impi",
                                         "module load   python3/3.9.2"
                                        ],
        "launch_methods"              : {
                                         "order"  : ["MPIRUN"],
                                         "MPIRUN" : {
                                             "pre_exec_cached": [
                                                 "module load TACC"
                                             ]
                                         }
                                        },
        
        "virtenv_mode"                : "local"
    }
}

The definition of each field:

* __description__ (_optional_) - human-readable description of the platform.
* __notes__ (_optional_) - information needed to form valid pilot descriptions, such as what parameters are required, etc.
* __schemas__ - allowed values for the `pd.access_schema` attribute of the pilot description. The first schema in the list is used by default. For each schema, a subsection is needed, which specifies __job_manager_endpoint__ and __filesystem_endpoint__.
   * __job_manager_endpoint__ - access URL for pilot submission (interpreted by PSI/J and RADICAL-SAGA).
   * __filesystem_endpoint__ - access URL for file staging (interpreted by PSI/J and RADICAL-SAGA).
* __default_queue__ (_optional_) - queue name to be used for pilot submission to a corresponding batch system (see __job_manager_endpoint__).
* __resource_manager__ - the type of job management system. Valid values are: `COBALT`, `FORK`, `LSF`, `PBSPRO`, `SLURM`.
* __cores_per_node__ (_optional_) - number of available CPU cores per compute node. If not provided then it will be discovered by Resource Manager in RADICAL-Pilot.
* __gpus_per_node__ (_optional_) - number of available GPUs per compute node. If not provided then it will be discovered by Resource Manager in RADICAL-Pilot.
* __system_architecture__ (_optional_) - set of options that describe platform features:
   * __smt__ - Simultaneous MultiThreading (i.e., threads per physical core). If it is not provided then the default value `1` is used. It could be reset with env variable `RADICAL_SMT` exported before running RADICAL-Pilot application. RADICAL-Pilot uses `cores_per_node x smt` to calculate all available cores/CPUs per node.
   * __options__ - list of job management system specific attributes/constraints, which are provided to PSI/J and RADICAL-SAGA.
      * `COBALT` uses option `--attrs` for configuring location as `filesystems=home,grand`, `mcdram` as `mcdram=flat`, `numa` as `numa=quad`;
      * `LSF` uses option `-alloc_flags` to support `gpumps`, `nvme`;
      * `PBSPRO` uses option `-l` for configuring location as `filesystems=grand:home`, placement as `place=scatter`;
      * `SLURM` uses option `--constraint` for compute nodes filtering.
   * __blocked_cores__ - list of cores/CPUs indices, which are not used by Scheduler in RADICAL-Pilot for tasks assignment.
   * __blocked_gpus__ - list of GPUs indices, which are not used by Scheduler in RADICAL-Pilot for tasks assignment.
* __agent_config__ - configuration file for RADICAL-Pilot Agent (default value is `default` for a corresponding file [agent_default.json](https://github.com/radical-cybertools/radical.pilot/blob/master/src/radical/pilot/configs/agent_default.json)).
* __agent_scheduler__ - Scheduler in RADICAL-Pilot (default value is `CONTINUOUS`).
* __agent_spawner__ - Executor in RADICAL-Pilot, which spawns task execution processes (default value is `POPEN`).
* __default_remote_workdir__ (_optional_) - directory for agent sandbox (see the tutorials [Getting Started](../getting_started.ipynb#Generated-Output) and [Staging Data with RADICAL-Pilot](staging_data.ipynb#Locations)). If not provided then the current directory is used (`$PWD`).
* __forward_tunnel_endpoint__ (_optional_) - name of the host, which can be used to create ssh tunnels from the compute nodes to the outside of the platform.
* __pre_bootstrap_0__ (_optional_) - list of commands to execute for the bootstrapping process to launch RADICAL-Pilot Agent.
* __pre_bootstrap_1__ (_optional_) - list of commands to execute for initialization of sub-agent, which are used to run additional instances of RADICAL-Pilot components such as Executor and Stager.
* __launch_methods__ - set of supported launch methods. Valid values are `APRUN`, `CCMRUN`, `FLUX`, `FORK`, `IBRUN`, `JSRUN` (`JSRUN_ERF`), `MPIEXEC` (`MPIEXEC_MPT`), `MPIRUN` (`MPIRUN_CCMRUN`, `MPIRUN_DPLACE`, `MPIRUN_MPT`, `MPIRUN_RSH`), `PRTE`, `RSH`, `SRUN`, `SSH`. For each launch method, a subsection is needed, which specifies __pre_exec_cached__ with list of commands to be executed to configure the launch method, and method related options (e.g., __dvm_count__ for `PRTE`).
   * __order__ - sets the order of launch methods to be selected for the task placement (the first value in the list is a default launch method).
* __python_dist__ - python distribution. Valid values are `default` and `anaconda`.
* __virtenv_mode__ - bootstrapping process set the environment for RADICAL-Pilot Agent (default value is `local`):
   * `create` - create a python virtual environment from scratch;
   * `recreate` - delete the existing virtual environment and build it from scratch, if not found then `create`;
   * `use` - use the existing virtual environment, if not found then `create`;
   * `update` - update the existing virtual environment, if not found then `create`;
   * `local` - use the client existing virtual environment (environment from where RADICAL-Pilot application was launched).
* __virtenv__ (_optional_) - path to the existing virtual environment or its name with the pre-installed RADICAL stack; use it only when `virtenv_mode=use`.
* __rp_version__ - RADICAL-Pilot installation or reuse process (default value is `installed`):
   * `local` - install from tarballs, from client existing environment;
   * `release` - install the latest released version from PyPI;
   * `installed` - do not install, target virtual environment has it.

## Examples

<div class="alert alert-info">

__Note:__ In our examples, we will not show a progression bar while waiting for some operation to complete, e.g., while waiting for a pilot to stop. That is because the progression bar offered by RP's reporter does not work within a notebook. You could use it when executing an RP application as a standalone Python script.

</div>

In [ ]:
%env RADICAL_REPORT=TRUE
%env RADICAL_REPORT_ANIME=FALSE

In [ ]:
# ensure that the location for user-defined configurations exists
!mkdir -p "${RADICAL_CONFIG_USER_DIR:-$HOME}/.radical/pilot/configs/"

In [ ]:
import os

import radical.pilot as rp
import radical.utils as ru

With the next steps, you will save the earlier created configuration for a target platform into the file `resource_tacc.json`, located in a user-space. You also will be able to read that file and print some of its attributes to confirm that they are in place.

In [ ]:
# save earlier defined platform configuration into the user-space
ru.write_json(resource_tacc_tutorial, 
              os.path.join(os.path.expanduser('~'), 
                           '.radical/pilot/configs/resource_tacc.json'))

In [ ]:
tutorial_cfg = rp.utils.get_resource_config(resource='tacc.frontera_tutorial')

for attr in ['schemas', 'resource_manager', 'cores_per_node', 'system_architecture']:
    print('%-20s : %s' % (attr, tutorial_cfg[attr]))

In [ ]:
print('job_manager_endpoint : ', 
      rp.utils.get_resource_job_url(resource='tacc.frontera_tutorial', schema='ssh'))
print('filesystem_endpoint  : ', 
      rp.utils.get_resource_fs_url (resource='tacc.frontera_tutorial', schema='ssh'))

In [ ]:
session = rp.Session()
pmgr    = rp.PilotManager(session=session)

In [ ]:
tutorial_cfg = session.get_resource_config(resource='tacc.frontera_tutorial', schema='local')
for attr in ['label', 'launch_methods', 'job_manager_endpoint', 'filesystem_endpoint']:
    print('%-20s : %s' % (attr, ru.as_dict(tutorial_cfg[attr])))

Platform description created above is also available within the :class:`rp.Session <radical.pilot.Session>` object. Let's confirm that newly created resource description is within the session. :class:`rp.Session <radical.pilot.Session>` object has all provided platform configurations (pre- and user-defined ones), thus for a pilot you just need to select a particular configuration and a corresponding access schema (as part of the pilot description).

In [ ]:
pd = rp.PilotDescription({
    'resource'     : 'tacc.frontera_tutorial',
    'project'      : 'XYZ000',
    'queue'        : 'production',
    'cores'        : 56,
    'runtime'      : 15,
    'exit_on_error': False,
    'access_schema': 'no_submission'  # user defined schema (see configuration file)
})

pilot = pmgr.submit_pilots(pd)

In [ ]:
from pprint import pprint

pprint(pilot.as_dict())

After exploring pilot setup and configuration we close the session.

In [ ]:
session.close()